In [ ]:
import tweepy as tw
import pandas as pd
from datetime import datetime, date, timedelta
import json
from GoogleCloud import GoogleCloud
import os
from google.cloud import storage
import time

In [ ]:
with open(r'D:\Estudos\FIAP\Acessos.json', 'r') as arquivo:
    acesso_api =  json.load(arquivo)

In [ ]:
cliente = tw.Client(bearer_token=acesso_api['bearer_token'], 
                    consumer_key=acesso_api['consumer_key'], 
                    consumer_secret=acesso_api['consumer_secret'],
                    access_token=acesso_api['access_token'],
                    access_token_secret=acesso_api['access_token_secret'])

In [ ]:
def busca_tweet(data_inicio, data_fim, intervalo,query):
    df_i = datetime.fromisoformat(data_inicio)
    df_f = datetime.fromisoformat(data_fim)

    lista_dados = []
    while (df_i < df_f):
        start = str(df_i).replace(' ','T')+'Z'
        df_i = df_i+timedelta(minutes=intervalo)-timedelta(seconds=1)
        end = str(df_i).replace(' ','T')+'Z'
        df_i = df_i+timedelta(seconds=1)
        lista_tweets = cliente.search_recent_tweets(query=query, 
                                                    max_results=100,
                                                    start_time=start,
                                                    end_time=end,
                                                    tweet_fields=['created_at',
                                                                    'public_metrics',
                                                                    'lang',
                                                                    'author_id',
                                                                    'in_reply_to_user_id'
                                                                    ])
        lista_dados.append(lista_tweets.data)
        print(start)
    return lista_dados

In [ ]:
lista_dados = busca_tweet('2022-03-24','2022-03-29',10,'studio_z')

In [ ]:
lista_dados.to_csv('traducao_01.csv' , index=False)

In [ ]:
resultado = []
for tweet_data in lista_dados:

    if tweet_data is not None and len(tweet_data) > 0:
        for tweet in tweet_data:
            obj = {
                    "id":tweet.id,
                    "author_id":tweet.author_id,
                    "created_at":str(tweet.created_at)[:19],
                    "text":tweet.text,
                    "lang":tweet.lang,
                    "in_reply_to_user_id":tweet.in_reply_to_user_id,
                    "retweet_count":tweet.public_metrics['retweet_count'],
                    "reply_count":tweet.public_metrics['reply_count'],
                    "like_count":tweet.public_metrics['like_count'],
                    "quote_count":tweet.public_metrics['quote_count']

                }
            resultado.append(obj)

In [ ]:
# with open('D:\Estudos\FIAP\Fase 5\dados.json', 'w') as arquivo:
#     arquivo.write(json.dumps(resultado, indent=4))

In [ ]:
# Conexão com o Cloud
cloud = GoogleCloud('D:\Estudos\FIAP\ServiceKey_Google_Cloud.json')
print(cloud.get_key())

In [ ]:
# Faz o Upload do arquivo JSON em Memoria para o Cloud

arquivo_json = json.dumps(resultado, indent=4)
bucket_name = 'data_lake_2022'
cliente = 'studio_z'
blob_name = 'dados.json'

cloud.upload_blob_from_memory(bucket_name,
                                arquivo_json,
                                cliente+'/'+blob_name )

In [ ]:
# Extrai um arquivos especifico de um diretorio do Cloud
bucket_name = 'data_lake_2022'
cliente = 'studio_z'
blob_name = 'dados.json'
aux = cloud.download_blob_to_json_list(bucket_name,cliente+'/'+blob_name)

In [ ]:
# Extrai todos os arquivos de um diretorio do Cloud
bucket_name = 'data_lake_2022'
cliente = 'studio_z'
aux2 = cloud.get_list_blob(bucket_name, cliente)

lista_arquivos = []
for arquivo in aux2:
    aux = cloud.download_blob_to_json_list(bucket_name,arquivo.name)
    lista_arquivos.append(aux)